In [43]:
import pandas as pd
import os

In [44]:
currentPath = os.getcwd()
dataPath = (currentPath + '/../data/external')


In [45]:
crspmret = pd.read_csv(dataPath + '/crspmret.csv')
crspminfo = pd.read_csv(dataPath + '/crspminfo.csv')

In [46]:
import numpy as np

# Split stocks in size according to ... 
*(10 and 20th percentile, check R code to see what it is exactly)*

In [47]:
# Extra from this notebook - may be needed for separate analyses
# What is me_nyse10?
conditions = [
    crspminfo['me'] <= crspminfo['me_nyse10'],
    (crspminfo['me'] >= crspminfo['me_nyse10']) & (crspminfo['me'] <= crspminfo['me_nyse20']),
    crspminfo['me'] > crspminfo['me_nyse20']
]

choices = ['Small', 'Medium', 'Big']

crspminfo['Size'] = np.select(conditions, choices, default=np.nan)

In [48]:
df = pd.read_csv(dataPath+'/signed_predictors_all_wide.csv', usecols=['permno','yyyymm','BMdec'])

In [49]:
crsp = crspminfo.merge(crspmret, on=['permno','yyyymm']).drop('date',axis=1)

In [50]:
crsp = crsp.merge(df, on=['permno', 'yyyymm']).dropna(subset=['BMdec'])

In [51]:
#crsp.dropna(subset=['BMdec'], inplace=True)

In [52]:
signaljune = crsp.loc[crsp['yyyymm']%100 == 6]

In [53]:
temp = signaljune.loc[(signaljune['shrcd'].isin([10,11])) & (signaljune['exchcd'] == 1)]

In [54]:
nysebreaks = temp.groupby('yyyymm')['BMdec'].apply(lambda x: x.quantile(q=[0.3,0.7])).reset_index().rename(columns={'level_1':'qlevel', 'BMdec': 'BMdec_breakpoint'})

In [55]:
sizebreaks = temp.groupby('yyyymm')['me'].apply(lambda x: x.quantile(q=0.5)).reset_index().rename(columns={'me':'me_0.5'})

In [56]:
sizebreaks

,yyyymm,me_0.5
0,195206,7.830000e+04
1,195306,6.221931e+04
2,195406,7.643131e+04
3,195506,1.491342e+05
4,195606,1.753388e+05
...,...,...
65,201706,5.322028e+07
66,201806,5.690366e+07
67,201906,6.600689e+07
68,202006,5.393771e+06


In [57]:
nysebreaks = nysebreaks.pivot(index='yyyymm', columns='qlevel', values='BMdec_breakpoint').rename(columns={0.3:'BMdec_0.3', 0.7:'BMdec_0.7'}).rename_axis(index='yyyymm', columns=None).reset_index()

In [58]:
port6 = signaljune.loc[(signaljune['exchcd'].isin([1,2,3])) & (signaljune['shrcd'].isin([10,11]))]
port6 = port6.merge(nysebreaks, on='yyyymm', how='left')
port6 = port6.merge(sizebreaks, on='yyyymm', how='left')

In [59]:
# Size Sort (Small and Big)
port6['FF_size_sort'] = np.select(
    [
        port6['me'] <= port6['me_0.5'],
        port6['me'] > port6['me_0.5']
    ],
    [
        'S',
        'B',
    ]
)

In [60]:
# Part of the HML sort (High, Medium, Low)
port6['BM_sort'] = np.select(
    [
        port6['BMdec'] <= port6['BMdec_0.3'],
        (port6['BMdec'] > port6['BMdec_0.3']) & (port6['BMdec'] <= port6['BMdec_0.7']),
        port6['BMdec'] > port6['BMdec_0.7']
    ],
    [
        port6['FF_size_sort']+'L',
        port6['FF_size_sort']+'M',
        port6['FF_size_sort']+'H'
    ]
)

In [61]:

#port6ret = 
port6.loc[np.floor(port6['yyyymm']/100) >= 1963][['permno','yyyymm','ret','melag','BM_sort']]


# Continue following the last part of the R script


,permno,yyyymm,ret,melag,BM_sort
0,10001,198806,-1.203884,6386.00000,SH
1,10001,198906,1.714286,6986.00000,SH
2,10001,199006,1.410256,10013.25000,SL
3,10001,199106,7.848101,10408.25000,SM
4,10001,199206,-0.770833,12900.00000,SH
...,...,...,...,...,...
2371,10279,198906,6.122449,344078.00000,SL
2372,10279,199006,-4.248366,544106.25000,SL
2373,10279,199106,-7.518797,483953.75000,SL
2374,10281,198906,1.639344,11332.65625,SL
